# Lab | API wrappers - Create your collection of songs & audio features

## Instructions

To move forward with the project, you need to create a collection of songs with their audio features - as large as possible!

These are the songs that we will cluster. And, later, when the user inputs a song, we will find the cluster to which the song belongs and recommend a song from the same cluster. The more songs you have, the more accurate and diverse recommendations you'll be able to give. Although... you might want to make sure the collected songs are "curated" in a certain way. Try to find playlists of songs that are diverse, but also that meet certain standards.

The process of sending hundreds or thousands of requests can take some time - it's normal if you have to wait a few minutes (or, if you're ambitious, even hours) to get all the data you need.

An idea for collecting as many songs as possible is to start with all the songs of a big, diverse playlist and then go to every artist present in the playlist and grab every song of every album of that artist. The amount of songs you'll be collecting per playlist will grow exponentially!

In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

import warnings
warnings.filterwarnings('ignore')

- Credentials 

In [2]:
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id="534dcca4cc9945aeb00fd619d64ddf59",
                                                           client_secret="ed4a166f4e484ce4b7feb1d76de6b0f0"))

In [3]:
results = sp.search(q='artist:Beyoncé', limit=50, type='track')  ### type!
results

{'tracks': {'href': 'https://api.spotify.com/v1/search?query=artist%3ABeyonc%C3%A9&type=track&offset=0&limit=50',
  'items': [{'album': {'album_type': 'single',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/6vWDO969PvNqNYHIOW5v0m'},
       'href': 'https://api.spotify.com/v1/artists/6vWDO969PvNqNYHIOW5v0m',
       'id': '6vWDO969PvNqNYHIOW5v0m',
       'name': 'Beyoncé',
       'type': 'artist',
       'uri': 'spotify:artist:6vWDO969PvNqNYHIOW5v0m'}],
     'available_markets': ['AD',
      'AE',
      'AG',
      'AL',
      'AM',
      'AO',
      'AR',
      'AT',
      'AU',
      'AZ',
      'BA',
      'BB',
      'BD',
      'BE',
      'BF',
      'BG',
      'BH',
      'BI',
      'BJ',
      'BN',
      'BO',
      'BR',
      'BS',
      'BT',
      'BW',
      'BY',
      'BZ',
      'CA',
      'CD',
      'CG',
      'CH',
      'CI',
      'CL',
      'CM',
      'CO',
      'CR',
      'CV',
      'CW',
      'CY',
      'CZ',
      'DE

In [4]:
#confirming number of songs
len(results['tracks']['items'])

50

In [5]:
#confirming we only have tracks and the features
results.keys()

dict_keys(['tracks'])

In [6]:
results["tracks"]["items"][0].keys()

dict_keys(['album', 'artists', 'available_markets', 'disc_number', 'duration_ms', 'explicit', 'external_ids', 'external_urls', 'href', 'id', 'is_local', 'name', 'popularity', 'preview_url', 'track_number', 'type', 'uri'])

In [7]:
results["tracks"]["items"][0]['name']

'AMERICA HAS A PROBLEM (feat. Kendrick Lamar)'

In [8]:
results["tracks"]["items"][15]['name']

'CUFF IT - WETTER REMIX'

In [9]:
results["tracks"]["items"][37]['name']

'Hold Up'

In [10]:
results["tracks"]["items"][48]['name']

'Love Drought'

- Focusing on a specific playlist

In [20]:
playlist = sp.user_playlist_tracks("spotify", "6CPtPfkNCud9O3PmVwntve", limit = 50) # searching for the playlist that has that id
playlist

{'href': 'https://api.spotify.com/v1/playlists/6CPtPfkNCud9O3PmVwntve/tracks?offset=0&limit=50&additional_types=track',
 'items': [{'added_at': '2020-11-10T08:53:45Z',
   'added_by': {'external_urls': {'spotify': 'https://open.spotify.com/user/glondoudou'},
    'href': 'https://api.spotify.com/v1/users/glondoudou',
    'id': 'glondoudou',
    'type': 'user',
    'uri': 'spotify:user:glondoudou'},
   'is_local': False,
   'primary_color': None,
   'track': {'album': {'album_type': 'album',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/26dSoYclwsYLMAKD3tpOr4'},
       'href': 'https://api.spotify.com/v1/artists/26dSoYclwsYLMAKD3tpOr4',
       'id': '26dSoYclwsYLMAKD3tpOr4',
       'name': 'Britney Spears',
       'type': 'artist',
       'uri': 'spotify:artist:26dSoYclwsYLMAKD3tpOr4'}],
     'available_markets': ['AD',
      'AE',
      'AG',
      'AL',
      'AM',
      'AO',
      'AR',
      'AT',
      'AU',
      'AZ',
      'BA',
      'BB',
     

In [21]:
playlist = sp.playlist("6CPtPfkNCud9O3PmVwntve")

In [22]:
playlist["name"]

"It's Britney Bitch"

In [26]:
#total of songs
results = sp.user_playlist_tracks("spotify", "6CPtPfkNCud9O3PmVwntve")
tracks = results['items']

while results['next']:
    results = sp.next(results)
    tracks.extend(results['items'])
len(tracks)

37

In [27]:
def get_playlist_tracks(username, playlist_id):
    
    results = sp.user_playlist_tracks(username, playlist_id)
    tracks = results['items']
    
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    
    return tracks

tracks = get_playlist_tracks("spotify", "6CPtPfkNCud9O3PmVwntve")

In [28]:
tracks[0]['track']['artists']#.keys()#['name']

[{'external_urls': {'spotify': 'https://open.spotify.com/artist/26dSoYclwsYLMAKD3tpOr4'},
  'href': 'https://api.spotify.com/v1/artists/26dSoYclwsYLMAKD3tpOr4',
  'id': '26dSoYclwsYLMAKD3tpOr4',
  'name': 'Britney Spears',
  'type': 'artist',
  'uri': 'spotify:artist:26dSoYclwsYLMAKD3tpOr4'}]

In [29]:
len(tracks)

37

- Trying with a different playlist, to get various artists

In [30]:
playlist2 = sp.user_playlist_tracks("spotify", "67bfHLzp1IjEZLJZUtuZVt", limit = 50) # searching for the playlist that has that id
playlist2

{'href': 'https://api.spotify.com/v1/playlists/67bfHLzp1IjEZLJZUtuZVt/tracks?offset=0&limit=50&additional_types=track',
 'items': [{'added_at': '2018-02-17T12:15:27Z',
   'added_by': {'external_urls': {'spotify': 'https://open.spotify.com/user/glondoudou'},
    'href': 'https://api.spotify.com/v1/users/glondoudou',
    'id': 'glondoudou',
    'type': 'user',
    'uri': 'spotify:user:glondoudou'},
   'is_local': False,
   'primary_color': None,
   'track': {'album': {'album_type': 'album',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/2UPnuV7os71xTZTyyEgj1B'},
       'href': 'https://api.spotify.com/v1/artists/2UPnuV7os71xTZTyyEgj1B',
       'id': '2UPnuV7os71xTZTyyEgj1B',
       'name': 'Steve Tyrell',
       'type': 'artist',
       'uri': 'spotify:artist:2UPnuV7os71xTZTyyEgj1B'}],
     'available_markets': ['AD',
      'AE',
      'AG',
      'AL',
      'AM',
      'AO',
      'AR',
      'AT',
      'AU',
      'AZ',
      'BA',
      'BB',
      '

In [31]:
playlist2 = sp.playlist("67bfHLzp1IjEZLJZUtuZVt")

In [33]:
playlist2["name"]

' 🍽'

In [34]:
#total of songs
results = sp.user_playlist_tracks("spotify", "67bfHLzp1IjEZLJZUtuZVt")
tracks = results['items']

while results['next']:
    results = sp.next(results)
    tracks.extend(results['items'])
len(tracks)

111

In [36]:
results = sp.user_playlist_tracks("spotify", "67bfHLzp1IjEZLJZUtuZVt")
tracks = results['items']

for oset in range(100,results['total'],100):
    print(oset)
    results = sp.user_playlist_tracks("spotify", "67bfHLzp1IjEZLJZUtuZVt", offset=oset)
    tracks += results['items']
len(tracks)

100


111

In [37]:
def get_playlist_tracks(username, playlist_id):
    
    results = sp.user_playlist_tracks(username, playlist_id)
    tracks = results['items']
    
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    
    return tracks

tracks = get_playlist_tracks("spotify", "67bfHLzp1IjEZLJZUtuZVt")

- Getting all the singers

In [39]:
def get_artists_and_ids_from_playlist(playlist_id):
    
    tracks_from_playlist = get_playlist_tracks("spotify", playlist_id)
    
    artists = []
    artists_ids = []
    
    for track in tracks_from_playlist:
        artists_info = track['track']['artists']
        
        for artist in artists_info:
            artists.append(artist['name'])
            artists_ids.append(artist['id'])
    
    return list(set(artists)), list(set(artists_ids))

In [40]:
artist_names, artist_ids = get_artists_and_ids_from_playlist("67bfHLzp1IjEZLJZUtuZVt")

In [41]:
artist_names

['Bill Withers',
 'Mary J. Blige',
 'Percy Sledge',
 'Wes Montgomery',
 'Nancy Sinatra',
 'Bobby Caldwell',
 'Ray Charles',
 'Anita Baker',
 'Corinne Bailey Rae',
 'Anthony Hamilton',
 'Earth, Wind & Fire',
 'Wilton Felder',
 'Tony Bennett',
 'Paul Anka',
 'Dinah Washington',
 'Bria Skonberg',
 'Brenna Whitaker',
 'Donnie',
 "The O'Jays",
 'The Impressions',
 'Dick Stabile And His Orchestra',
 'Achille Togliani',
 'Marilyn Monroe',
 'Sam Cooke',
 'Dawn Penn',
 'Donny Hathaway',
 'Patti LaBelle',
 'Michael Bublé',
 'Amy Winehouse',
 'James Ingram',
 'The Chi-Lites',
 'Leonard Cohen',
 'Curtis Mayfield',
 'The Real Thing',
 'Nicole Henry',
 'Louis Armstrong',
 'Steve Tyrell',
 'Billy Paul',
 'Erykah Badu',
 'Grover Washington, Jr.',
 'Roy Orbison',
 'Charles & Eddie',
 'Etta James',
 'Norah Jones',
 'The Temprees',
 'Jolie Holland',
 'Santana',
 'Matt Dusk',
 'Walter Hawkins',
 'Dean Martin',
 'Joss Stone',
 'Nat King Cole',
 'Charles Bradley',
 'Ella Fitzgerald',
 'Frank Sinatra',
 'Sar

- Getting the song titles

In [42]:
tracks = get_playlist_tracks("spotify", "67bfHLzp1IjEZLJZUtuZVt")
track_names = [track['track']['name'] for track in tracks]
track_names

['Sunday Kind of Love',
 'Like Someone in Love',
 'My Funny Valentine',
 'Body and Soul',
 'At Last - Single Version',
 'Little Paradise',
 'La vie en rose',
 'Make It Last',
 "On an Evening in Roma (Sott'er Celo de Roma)",
 "I'm Old Fashioned",
 'Let’s Fall In Love',
 'Deep in a Dream',
 'Time Alone',
 'Woman',
 'But Beautiful',
 'Best of Me',
 'Say a Little Prayer - Live',
 'O-o-h Child',
 'Feeling Good',
 'Dance Me to the End of Love',
 "Let's Stay Together",
 'Old Fashioned Morphine',
 "You Don't Own Me",
 'I Try',
 'Baby, This Love I Have',
 'Heaven Sent',
 'Broke Down Piece of Man',
 'These Arms of Mine',
 'Wonderful World',
 "Let's Get It On",
 'My Girl',
 'Now That We Found Love',
 "Ain't No Sunshine",
 'Oh Girl',
 'L-O-V-E - Remastered',
 'A Sunday Kind Of Love - Single Version',
 'Me and Mrs. Jones',
 'People Get Ready',
 'Stand by Me',
 'Fly Me To The Moon - Remastered 2008',
 'I Say a Little Prayer',
 "I'd Rather Go Blind - Single Version",
 'A Whole Lotta Woman',
 'When I 

- Prototype

In [44]:
#defining track_ids first as I got an error
tracks = get_playlist_tracks("spotify", "67bfHLzp1IjEZLJZUtuZVt")
track_ids = [track['track']['id'] for track in tracks]

In [45]:
sp.audio_features(track_ids[0:100])

[{'danceability': 0.579,
  'energy': 0.338,
  'key': 10,
  'loudness': -12.014,
  'mode': 1,
  'speechiness': 0.0333,
  'acousticness': 0.836,
  'instrumentalness': 7.83e-06,
  'liveness': 0.154,
  'valence': 0.436,
  'tempo': 113.129,
  'type': 'audio_features',
  'id': '6XmN8QIxuZt8ljcnaPmhOM',
  'uri': 'spotify:track:6XmN8QIxuZt8ljcnaPmhOM',
  'track_href': 'https://api.spotify.com/v1/tracks/6XmN8QIxuZt8ljcnaPmhOM',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/6XmN8QIxuZt8ljcnaPmhOM',
  'duration_ms': 211360,
  'time_signature': 3},
 {'danceability': 0.464,
  'energy': 0.319,
  'key': 8,
  'loudness': -12.075,
  'mode': 1,
  'speechiness': 0.0449,
  'acousticness': 0.89,
  'instrumentalness': 0.000336,
  'liveness': 0.111,
  'valence': 0.391,
  'tempo': 146.521,
  'type': 'audio_features',
  'id': '1my1nf6O10GIZEZRp2Cxv7',
  'uri': 'spotify:track:1my1nf6O10GIZEZRp2Cxv7',
  'track_href': 'https://api.spotify.com/v1/tracks/1my1nf6O10GIZEZRp2Cxv7',
  'analysis_url': 'ht